<a href="https://colab.research.google.com/github/skagnihotri/MALE-FEMALE-CLASSIFICATION/blob/master/male_female_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MALE FEMALE CLASSIFICATION

## Loading Data

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smagni","key":"b4fd8d61cfa7a806a19e09267e4f31d6"}'}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d playlist/men-women-classification

 98% 597M/606M [00:05<00:00, 114MB/s]
100% 606M/606M [00:06<00:00, 105MB/s]


In [7]:
!unzip /content/men-women-classification.zip

Archive:  /content/men-women-classification.zip
replace men/00000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
from shutil import move

In [9]:
!mkdir training
move("men", "training")
move("women", "training")

'training/women'

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

In [13]:
generator = datagen.flow_from_directory('/content/training',
                                              target_size=(64, 64),
                                              batch_size= 3309,
                                              class_mode='binary')

Found 3309 images belonging to 2 classes.


In [0]:
X, y = generator.next()

In [15]:
X.shape, y.shape

((3309, 64, 64, 3), (3309,))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)
X_train.shape, X_test.shape

((2647, 64, 64, 3), (662, 64, 64, 3))

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input
from tensorflow.keras.models import Model

In [18]:
input = Input(shape= (64, 64, 3))
conv1 = Conv2D(filters= 64, kernel_size= (3, 3), strides= 1, padding= "same", activation= 'relu')(input)
pool1 = MaxPooling2D(pool_size= (2, 2), strides= 2, padding= "same")(conv1)
conv2 = Conv2D(filters= 64, kernel_size= (3, 3), strides= 1, padding= "same", activation= 'relu')(pool1)
pool2 = MaxPooling2D(pool_size= (2, 2), strides= 2, padding= "same")(conv2)

flatten = Flatten()(pool2)
hidden1 = Dense(units= 1024, activation= "relu")(flatten)
hidden2 = Dense(units= 512, activation= "relu")(hidden1)
output = Dense(units= 1, activation= "sigmoid")(hidden2)
model = Model(inputs = [input], outputs = [output])

W0809 16:39:50.181933 139723236562816 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics=["accuracy"])

W0809 16:40:03.342057 139723236562816 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
model.fit(X_train, y_train, batch_size= 25, epochs= 25, validation_split= 0.1)

Train on 2382 samples, validate on 265 samples
Epoch 1/25
2382/2382 [==============================] - 40s 17ms/sample - loss: 0.7025 - acc: 0.5617 - val_loss: 0.6845 - val_acc: 0.5887
Epoch 2/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.6597 - acc: 0.6125 - val_loss: 0.6923 - val_acc: 0.5170
Epoch 3/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.6230 - acc: 0.6524 - val_loss: 0.6583 - val_acc: 0.6226
Epoch 4/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.5609 - acc: 0.7149 - val_loss: 0.6570 - val_acc: 0.6415
Epoch 5/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.4590 - acc: 0.7800 - val_loss: 0.6997 - val_acc: 0.6528
Epoch 6/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.3380 - acc: 0.8518 - val_loss: 0.9588 - val_acc: 0.6528
Epoch 7/25
2382/2382 [==============================] - 39s 16ms/sample - loss: 0.2105 - acc: 0.9165 - val_loss: 1.4183 - val

In [21]:
model.evaluate(X_test, y_test)

662/662 [==============================] - 2s 4ms/sample - loss: 3.0184 - acc: 0.6767


[3.0183589714171304, 0.6767372]